# Mapear automaticamente as tabelas do Postgres com Python

Código que cria um dicionário de dados das tabelas existentes do schema público de uma database fictícia

 *Importações dos módulos*

In [1]:
import psycopg2
from psycopg2.extensions import AsIs

import functools 
import operator 

import json

*Definição das variáveis para acesso ao banco de dados*

In [2]:
pg_dbname = "dvdrental"  # nome database 
pg_username = "postgres" # nome do usuário
pg_password = "abc123"   # senha do usuário 

conn = psycopg2.connect(dbname=pg_dbname, user=pg_username, password=pg_password)
cur = conn.cursor() # Objeto de conexão

*Função que define as tabelas existentes no schema public da base de dados informada anteriormente*

In [3]:
def tables_names(cur):
    cur.execute("SELECT  table_name from information_schema.tables WHERE table_schema='public' ORDER BY table_name")
    table_names = cur.fetchall()
    
    return table_names


*Função que imprime as descrições das tabelas do schema public*

In [4]:
def tables_descriptions(cur):

    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    for table in cur.fetchall(): 
        table = table[0]
        cur.execute("SELECT * FROM %s LIMIT 0",    [AsIs(table)])
        print(cur.description, "\n")


In [ ]:
#tables_descriptions(cur)

*Função que converte lista de tuplas em lista de string*

In [6]:
def convertTuple(tup): 
    str =  functools.reduce(operator.add, (tup))
    return str


### Função que define as descrições das tabelas com nome, tipo de dado e tamanho em formato quase legível

In [7]:
def tables_readable_description(cur, table_list):

    # Dicionário com os tipos de dados do Postgres    
    cur.execute("SELECT oid, typname FROM pg_catalog.pg_type")
    type_mappings = {
        int(oid): typname
        for oid, typname in cur.fetchall()
    }    


    readable_description = {}
    for table in table_list:
        cur.execute("SELECT * FROM %s LIMIT 0", [AsIs(table)])
        readable_description[table] = dict(
            columns=[
                dict(
                    name=col.name,
                    type=type_mappings[col.type_code],
                    length=col.internal_size
                )
                for col in cur.description
            ]
        )
    
    return readable_description
    

*Função que adiciona a quantidade de linhas das tabelas*

In [8]:
def add_rows_total(cur, readable_description):
    for table in readable_description.keys():
        cur.execute("SELECT COUNT(*) FROM %s", [AsIs(table)])
        readable_description[table]["total"] = cur.fetchone()[0]
    
    return readable_description

## Chamada das funções

In [9]:
table_names = tables_names(cur) # Definimos uma lista com os nomes das tabelas

In [ ]:
#print("Quantidade de tabelas: ", len(table_names))
 
#for line in table_names:
#    print(line)

In [10]:
table_list = convertTuple(table_names) # Convertemos a lista de tuplas para uma lista de strings

readable_description = tables_readable_description(cur, table_list) # Criamos um dicionário de dados das tabelas

print(readable_description)

{'account': {'columns': [{'name': 'user_id', 'type': 'int4', 'length': 4}, {'name': 'username', 'type': 'varchar', 'length': 50}, {'name': 'password', 'type': 'varchar', 'length': 50}, {'name': 'email', 'type': 'varchar', 'length': 300}, {'name': 'created_on', 'type': 'timestamp', 'length': 8}, {'name': 'last_login', 'type': 'timestamp', 'length': 8}]}, 'actor': {'columns': [{'name': 'actor_id', 'type': 'int4', 'length': 4}, {'name': 'first_name', 'type': 'varchar', 'length': 45}, {'name': 'last_name', 'type': 'varchar', 'length': 45}, {'name': 'last_update', 'type': 'timestamp', 'length': 8}]}, 'actor_info': {'columns': [{'name': 'actor_id', 'type': 'int4', 'length': 4}, {'name': 'first_name', 'type': 'varchar', 'length': 45}, {'name': 'last_name', 'type': 'varchar', 'length': 45}, {'name': 'film_info', 'type': 'text', 'length': -1}]}, 'address': {'columns': [{'name': 'address_id', 'type': 'int4', 'length': 4}, {'name': 'address', 'type': 'varchar', 'length': 50}, {'name': 'address2',

In [11]:
add_rows_total(cur, readable_description) # Adiciona o total de linhas de cada tabela

json_str = json.dumps(readable_description, indent=2) # Converte o dicionário de dados para um formato legível em JSON 

print(json_str)

{
  "account": {
    "columns": [
      {
        "name": "user_id",
        "type": "int4",
        "length": 4
      },
      {
        "name": "username",
        "type": "varchar",
        "length": 50
      },
      {
        "name": "password",
        "type": "varchar",
        "length": 50
      },
      {
        "name": "email",
        "type": "varchar",
        "length": 300
      },
      {
        "name": "created_on",
        "type": "timestamp",
        "length": 8
      },
      {
        "name": "last_login",
        "type": "timestamp",
        "length": 8
      }
    ],
    "total": 0
  },
  "actor": {
    "columns": [
      {
        "name": "actor_id",
        "type": "int4",
        "length": 4
      },
      {
        "name": "first_name",
        "type": "varchar",
        "length": 45
      },
      {
        "name": "last_name",
        "type": "varchar",
        "length": 45
      },
      {
        "name": "last_update",
        "type": "timestamp",
       

In [12]:
conn.close() # Fechamos a conexão com o Postgres